In [ ]:

confusion_matrix_map = {'tp':'true positive', 'fp':'false positive', 'tn':'true negative', 'fn':'false negative'}


In [ ]:
test_genome_ids = pd.read_csv('../data/campylobacterota_dataset_test.csv', index_col=0).genome_id.unique()
test_results = Results(test_genome_ids, load_ref=True, load_labels=True)
test_results.load_predict('../data/predict/campylobacterota_dataset_test_predict.csv', model_name=model_name)


In [ ]:

def table_1(path:str='../data/figures/results-campylobacterota-table_1.png'):

    model_names = ['campylobacterota_esm_650m_gap_v1', 'campylobacterota_esm_650m_gap_v2', 'campylobacterota_esm_650m_gap_v3']
    test_results = Results(test_genome_ids, load_ref=True, load_labels=True)
    test_results.load_predict('../data/predict/campylobacterota_dataset_test_predict.csv', model_name=None)
    df = test_results.to_df()
    df = df.dropna()
    df['label'] = [0 if (label == 'spurious') else 1 for label in df.label]
    
    table_df = list()
    for model_name in model_names:
        row = dict()
        row['feature type'] = re.search('(esm_650m_gap|pt5_3b_gap|esm_3b_gap)', model_name).group(1) 
        row['model_name'] = model_name
        row.update(df[f'{model_name}_confusion_matrix'].replace(confusion_matrix_map).value_counts().to_dict())
        row['accuracy'] = np.round(balanced_accuracy_score(df.label, df[f'{model_name}_label']), 4)
        table_df.append(row)
    table_df = pd.DataFrame(table_df).set_index('model_name')
    table_df = table_df.sort_values('false positive')

    dfi.export(table_df, path, table_conversion='matplotlib')
    return table_df

table_3()
